In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mushroom/Dataset/test/Lactarius/0340_0gzvFhGVf5M.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0399_uO9IdWMscC4.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0625_tNFr4syCPSs.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0405_XQ5FsiWx3Sc.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0508_-5qUGfoNfh8.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0099_DXVPtL-oMIU.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/162_smJc1hV6gjs.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/173_4DhN-XTjnZc.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/121_nXIG8vSAk_s.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0509_p7vbAEkoygw.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/029_Qx9sNkJyN9Q.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/309_OTtxvJRv6X4.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0162_DiWMr6h-ujY.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0529_zNaDx9ppi8Y.jpg
/kaggle/input/mushroom/Dataset/test/Lactarius/0278_xasbi9piTJU.jpg


In [4]:
# Core Python Libraries
import os
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow / Keras (Deep Learning)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [5]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,              # normalize pixel values
    rotation_range=30,              # random rotations
    width_shift_range=0.2,          # horizontal shift
    height_shift_range=0.2,         # vertical shift
    zoom_range=0.2,                 # zoom in/out
    brightness_range=[0.8, 1.2],    # vary brightness
    horizontal_flip=True,           # random horizontal flips
    vertical_flip=True              # random vertical flips
)


test_datagen = ImageDataGenerator(rescale=1.0/255.0)


train_dir = "/kaggle/input/mushroom/Dataset/train"
test_dir  = "/kaggle/input/mushroom/Dataset/test"

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),   # resize images
    batch_size=32,
    class_mode="binary"       # use 'categorical' if more than 2 classes
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)


Found 911 images belonging to 3 classes.
Found 306 images belonging to 3 classes.


In [8]:
# ============================================
# Configure the Learning Process
# ============================================

# Adam optimizer with a small learning rate for fine-tuning
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',  # since we are using softmax in last layer
    metrics=['accuracy']              # accuracy as performance metric
)


In [14]:
x = Flatten()(x)
output_layer = Dense(1, activation='sigmoid')(x)   # 1 output neuron
model = Model(inputs=base_model.input, outputs=output_layer)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [16]:
train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/mushroom/Dataset/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'    # must match sigmoid
)

validation_generator = test_datagen.flow_from_directory(
    "/kaggle/input/mushroom/Dataset/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'    # must match sigmoid
)


Found 911 images belonging to 3 classes.
Found 306 images belonging to 3 classes.


In [10]:
# Use test_generator as validation data
validation_generator = test_generator


In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# =========================
# 1. Data Preparation
# =========================

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/mushroom/Dataset/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'   # ✅ categorical for 3 classes
)

# Validation/Test generator
validation_generator = test_datagen.flow_from_directory(
    "/kaggle/input/mushroom/Dataset/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'   # ✅ categorical for 3 classes
)

# =========================
# 2. Pre-trained CNN (Feature Extractor)
# =========================

base_model = Xception(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze convolutional base
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
output_layer = Dense(train_generator.num_classes, activation="softmax")(x)  # ✅ 3 classes

model = Model(inputs=base_model.input, outputs=output_layer)

# =========================
# 3. Compile Model
# =========================

model.compile(optimizer="adam",
              loss="categorical_crossentropy",  # ✅ categorical loss
              metrics=["accuracy"])

# =========================
# 4. Train Model
# =========================

steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# =========================
# 5. Model Summary
# =========================
model.summary()


Found 911 images belonging to 3 classes.
Found 306 images belonging to 3 classes.
Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 29s 740ms/step - accuracy: 0.5323 - loss: 4.3454 - val_accuracy: 0.7778 - val_loss: 1.2877
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.7812 - loss: 1.4682 - val_accuracy: 0.7431 - val_loss: 1.7472
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 15s 546ms/step - accuracy: 0.7954 - loss: 1.0509 - val_accuracy: 0.7812 - val_loss: 1.5781
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.7500 - loss: 2.7107 - val_accuracy: 0.8368 - val_loss: 0.9277
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 554ms/step - accuracy: 0.8454 - loss: 0.9151 - val_accuracy: 0.9062 - val_loss: 0.3613
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.9375 - loss: 0.5823 - val_accuracy: 0.8993 - val_loss: 0.3557
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 551ms/step - accuracy: 0.8993 - loss: 0.5544 - val_accuracy: 0.9132 - val_loss: 0.3737
Epoch 8/20
28/28 ━━━━

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 111, 111,  │        864 │ input_layer_3[0]… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 111, 111,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 111, 111,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 109, 109,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 109, 109,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 109, 109,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 109, 109,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 109, 109,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 109, 109,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 109, 109,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 109, 109,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 55, 55,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 55, 55,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        512 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_36 (Add)        │ (None, 55, 55,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 55, 55,    │          0 │ add_36[0][0]    

 Total params: 21,764,659 (83.03 MB)

 Trainable params: 301,059 (1.15 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

 Optimizer params: 602,120 (2.30 MB)

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# =========================
# 1. Data Preprocessing
# =========================
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/mushroom/Dataset/train",   # <-- update with your dataset path
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

# Validation generator
valid_generator = train_datagen.flow_from_directory(
    "/kaggle/input/mushroom/Dataset/train",   # same folder, but subset=validation
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

# =========================
# 2. Define steps
# =========================
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

print("✅ Data generators created successfully!")


Found 730 images belonging to 3 classes.
Found 181 images belonging to 3 classes.
✅ Data generators created successfully!


In [31]:
import numpy as np
from tensorflow.keras.preprocessing import image

# ✅ Get class labels from training generator
class_labels = list(train_generator.class_indices.keys())
print("Class labels:", class_labels)

# =========================
# Predict on a single image
# =========================
img_path = "/kaggle/input/mushroom/Dataset/train/Boletus/0001_yB5GiXfgyRU.jpg"  # 👈 replace with your test image

# Load and preprocess image
img = image.load_img(img_path, target_size=(224, 224))   # must match model input size
img_array = image.img_to_array(img) / 255.0             # rescale like in training
img_array = np.expand_dims(img_array, axis=0)           # add batch dimension

# Predict
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)[0]
confidence = np.max(predictions)

print(f"✅ Predicted Class: {class_labels[predicted_class]} (Confidence: {confidence*100:.2f}%)")


Class labels: ['Boletus', 'Lactarius', 'Russula']
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
✅ Predicted Class: Boletus (Confidence: 99.87%)


In [29]:
from tensorflow.keras.models import load_model

# Load the saved best model
best_model = load_model("/kaggle/working/best_model.h5")

# Evaluate on validation set
loss, acc = best_model.evaluate(validation_generator, verbose=1)
print(f"📊 Validation Accuracy: {acc*100:.2f}%")
print(f"📉 Validation Loss: {loss:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 861ms/step - accuracy: 0.9201 - loss: 0.4434
📊 Validation Accuracy: 93.14%
📉 Validation Loss: 0.3635
